# Module 4 - RAG-Chatbot 

In this module we create a chatbot using RAG ([Retrieval Augmented Generation](https://en.wikipedia.org/wiki/Retrieval-augmented_generation)) and [GraphRAG](https://graphrag.com/). 

Import our usual suspects (and some more...)

In [1]:
import pandas as pd
import os
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from neo4j import Query, GraphDatabase, RoutingControl, Result
from dotenv import load_dotenv
import gradio as gr
import time
from IPython.display import display, HTML
import warnings
from json import loads, dumps
warnings.filterwarnings('ignore')

## Get Credentials

Load env variables

In [2]:
env_file = 'ws.env'

In [3]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
    LLM = os.getenv('LLM')
    EMBEDDINGS_MODEL = os.getenv('EMBEDDINGS_MODEL')
else:
    print(f"File {env_file} not found.")

## Setup Connection to Database

Setup connection to the database with the [Python Driver](https://neo4j.com/docs/python-manual/5/).

In [4]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)

Test the connection

In [5]:
driver.execute_query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

,Count
0,1388


## Create RAG-application

For the the chatbot we both need an Embedding-model and LLM. Create both below:

In [6]:
embedding_model = OpenAIEmbeddings(
    model=EMBEDDINGS_MODEL,
    openai_api_key=OPENAI_API_KEY
)

In [7]:
embedding_model.model

'text-embedding-ada-002'

In [8]:
llm = ChatOpenAI(temperature=0, model=LLM)

In [9]:
llm.model_name

'gpt-4o'

### Retrieval Queries

To illustrate the difference between a "Regular" Vector Search and GraphRAG we create different retrieval queries.

The following function retrieves the context using a regular vector search. 

In [10]:
def get_context_vector_search(search_prompt):
    query_vector = embedding_model.embed_query(search_prompt)
    
    similarity_query = """ 
        CALL db.index.vector.queryNodes("chunk-embeddings", 3, $query_vector) YIELD node, score
        WITH node as chunk, score ORDER BY score DESC
        MATCH (d:Document)<-[:PART_OF]-(chunk)
        RETURN score, d.file_name as file_name, chunk.id as chunk_id, chunk.page as page, chunk.chunk_eng AS chunk
       """
    results = driver.execute_query(
        similarity_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        query_vector=query_vector,
        result_transformer_= lambda r: r.to_df()
    )
    
    results = results.to_json(orient="records")
    parsed = loads(results)
    context = dumps(parsed, indent=2)

    return context

The following function retrieves the context using a the Knowledge Graph (GraphRAG). We start with a regular vector search and can find more than that. 

In [11]:
def get_context_graphrag(search_prompt):
    query_vector = embedding_model.embed_query(search_prompt)
    
    similarity_query = """ 
        CALL db.index.vector.queryNodes("chunk-embeddings", 3, $query_vector) YIELD node, score
        WITH node as chunk, score ORDER BY score DESC
        CALL (chunk) {
            MATCH (chunk)-[r:OVERLAPPING_DEFINITIONS]-(overlapping_chunk:Chunk)
            WHERE r.overlap > 3
            RETURN collect(overlapping_chunk) AS overlapping_chunks
        }
        WITH [chunk] + overlapping_chunks AS chunks
        UNWIND chunks as chunk
        MATCH (d:Document)<-[:PART_OF]-(chunk)
        RETURN d.file_name as file_name, chunk.id as chunk_id, chunk.page as page, chunk.chunk_eng AS chunk
       """
    results_1 = driver.execute_query(
        similarity_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        query_vector=query_vector,
        result_transformer_= lambda r: r.to_df()
    )

    chunk_ids = list(set(results_1['chunk_id'].to_list()))

    definition_query = """    
       CALL db.index.vector.queryNodes("definition-embeddings", 5, $query_vector) YIELD node, score
            WITH node as definition, score ORDER BY score DESC
            WHERE definition.degree < 20
            WITH definition LIMIT 1
            MATCH (definition)<-[:MENTIONS]-(chunk:Chunk)
            WHERE NOT (chunk.id IN $chunk_ids)
            WITH chunk LIMIT 3
            MATCH (d:Document)<-[:PART_OF]-(chunk)
            RETURN d.file_name as file_name, chunk.id as chunk_id, chunk.page as page, chunk.chunk_eng AS chunk
    """
    results_2 = driver.execute_query(
        definition_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        chunk_ids=chunk_ids,
        query_vector=query_vector,
        result_transformer_= lambda r: r.to_df()
    )
    results = pd.concat([results_1,results_2]).drop_duplicates()
    results = results.to_json(orient="records")
    parsed = loads(results)
    context = dumps(parsed, indent=2)
    return context

### Prompts 

Prompt for vector search which returns just the context.

In [12]:
def generate_prompt(search_prompt, context):
    prompt_template = """

    You are a chatbot on Rabobank product. Your goal is to help people with questions on product policies.  
    A user will come to you with questions on their policy. Their questions must be answered based on the relevant documents of the policy.
    Respond in English. 

    The question is the following: 
    {search_prompt}
    
    Always respond in the language in which the question was asked. So, do not respond in a different language.
    
    The context is the following: 
    {context}

    Please explain your answer as thorough as possbile based on the context above. Don't come up with anything yourself.
    
    Please end your message with listing your sources with file name and page number. 
    """
    prompt = PromptTemplate.from_template(prompt_template)
    
    theprompt = prompt.format_prompt(search_prompt=search_prompt, context=context)
    return theprompt

The prompt for GraphRAG provides context and definitions.

## Some examples to test the models

For every example there can be chosen between GraphRAG and vector search. 

In [13]:
search_prompt = 'What is meant with the Rabofoon?'

context = get_context_vector_search(search_prompt)
theprompt = generate_prompt(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

================================== Ai Message ==================================

The term "Rabofoon" refers to a service provided by Rabobank that allows customers to perform banking transactions over the phone. Specifically, it enables users to give payment orders using the phone's keys. Once a payment order is given, the user must follow the instructions provided by Rabofoon to authorize the transaction. The payment order is considered received by Rabobank as soon as the system records the confirmation key being pressed, at which point the order cannot be withdrawn. For transactions made via Rabofoon, the International Bank Account Number (IBAN) is used as a unique identifier. If a customer enters an account number, Rabobank will convert it into the beneficiary's IBAN, and the customer must verify that this is correct before the payment is executed.

Additionally, Rabofoon requires a personal access code for checking balances, transactions, or making transfers. There are no subscrip

In [14]:
search_prompt = 'What is meant with the Rabofoon?'

context = get_context_graphrag(search_prompt)
theprompt = generate_prompt(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

================================== Ai Message ==================================

The term "Rabofoon" refers to a telephone banking service provided by Rabobank. It allows users to perform certain banking activities over the phone. Here are the key points about Rabofoon based on the provided context:

1. **Payment Orders**: If you have access to Rabofoon and it is activated for you, you can give payment orders using the phone's keys. You authorize the payment order by following the instructions provided by Rabofoon. Once you press the confirmation key, the payment order is received by Rabobank's systems and cannot be withdrawn. For transfers, the IBAN is used as a unique identifier, and Rabobank will convert any account number you enter into the beneficiary's IBAN, which you must verify (Payment and Online Services Terms Sept 2022.pdf, page 51).

2. **Security Codes**: Rabofoon uses a 5-digit code for security, which can be blocked by entering an incorrect code three times if you suspe

In [15]:
search_prompt = 'Are you insured when traveling to a high-risk country?'

context = get_context_vector_search(search_prompt)
theprompt = generate_prompt(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

================================== Ai Message ==================================

Based on the provided context from the Interpolis Short-Term Travel Insurance policy documents, the insurance coverage depends on several factors, including the geographical area covered by your policy and the nature of your trip.

1. **Geographical Coverage**: The insurance applies either within Europe or worldwide, as specified in your policy document. The European coverage includes specific regions such as the Azores, the Canary Islands, Cyprus, Madeira, and parts of Russia. Outside of Europe, it also covers countries like Algeria, Egypt, Iceland, Israel, Lebanon, Libya, Morocco, Syria, Tunisia, and Turkey. However, it explicitly does not cover Bonaire, Sint Eustatius, and Saba (Interpolis Short-Term Travel Insurance, page 16).

2. **Nature of the Trip**: The insurance covers trips for leisure, volunteer work, holiday work, study, internship, and necessary private trips. However, it does not cover trip

In [16]:
search_prompt = 'Are you insured when traveling to a high-risk country?'

context = get_context_graphrag(search_prompt)
theprompt = generate_prompt(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

================================== Ai Message ==================================

Based on the provided context from the "Interpolis Short-Term Travel Insurance" document, you are not insured when traveling to a high-risk country if the area has been issued a 'red' color code by the Ministry of Foreign Affairs. The document explicitly states that assistance and costs are never covered if the insured travels to an area with a 'red' color code. However, if the insured is already in the area at the time it receives a 'red' color code, they are covered under the condition that they leave the area as soon as possible. It is important to note that the costs to leave the area are not covered.

This means that if you plan to travel to a high-risk country that has been designated with a 'red' color code, your travel insurance will not cover you. If you are already in such an area when the designation is made, you must leave promptly to maintain any coverage, but you will have to bear the costs 

In [17]:
search_prompt = 'What are the rules for a joint investment account?'

context = get_context_vector_search(search_prompt)
theprompt = generate_prompt(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

================================== Ai Message ==================================

The rules for a joint investment account at Rabobank, as outlined in the provided context, are as follows:

1. **Communication and Information Sharing**: Rabobank is only required to inform one account holder about any information related to the account. It is the responsibility of the informed account holder to immediately share this information with the other account holders. All account holders are bound by the information provided to any one of them (Rabo Beleggersrekening Terms 2020.pdf, page 5).

2. **Actions Upon Death of an Account Holder**: If one of the account holders passes away, the heirs of the deceased can only use the investment account together. They are also required to perform any legal actions related to the agreement and/or the general terms and conditions collectively (Rabo Beleggersrekening Terms 2020.pdf, page 5).

3. **Joint Use and Legal Actions**: For a joint investment account,

In [18]:
search_prompt = 'What are the rules for a joint investment account?'

context = get_context_graphrag(search_prompt)
theprompt = generate_prompt(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

================================== Ai Message ==================================

The rules for a joint investment account, as outlined in the provided context, are as follows:

1. **Joint and/or Investment Account vs. Joint Investment Account**: If the investment account has multiple account holders, it is considered a joint and/or investment account unless specifically agreed upon as a joint investment account (Rabo Beleggersrekening Terms 2020, page 5).

2. **Communication**: For both types of accounts, the bank only needs to inform one account holder. It is the responsibility of the informed account holder to immediately share any information with the other account holders. All account holders are bound by the information provided to any one of them (Rabo Beleggersrekening Terms 2020, page 5).

3. **Actions and Usage**: 
   - For a joint investment account, all account holders must use the account together and perform any legal actions regarding the agreement and/or general terms a

## Gradio Chatbot that uses RAG and GraphRAG

Example code is coming from Gradio documentation: [Creating a custom chatbot with blocks](https://www.gradio.app/guides/creating-a-custom-chatbot-with-blocks#add-streaming-to-your-chatbot)

In [19]:
def user(user_message, history):
    return "", history + [[user_message, None]]

def get_answer(search_prompt, rag_method):
    if rag_method == "Vector-Search":
        context = get_context_vector_search(search_prompt)
        theprompt = generate_prompt(search_prompt, context)
    else: 
    # rag_method == "GraphRAG"
        context = get_context_graphrag(search_prompt)
        theprompt = generate_prompt(search_prompt, context)
    messages = llm(theprompt.to_messages())
    return messages.content

def bot(history, rag_method):
    bot_message = get_answer(history[-1][0], rag_method)
    history[-1][1] = ""
    for character in bot_message:
        history[-1][1] += character
        time.sleep(0.01)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        label="Chatbot with RAG", 
        avatar_images=["https://png.pngtree.com/png-vector/20220525/ourmid/pngtree-concept-of-facial-animal-avatar-chatbot-dog-chat-machine-illustration-vector-png-image_46652864.jpg","https://d-cb.jc-cdn.com/sites/crackberry.com/files/styles/larger/public/article_images/2023/08/openai-logo.jpg"]
    )
    msg = gr.Textbox(label="Message")
    rag_method = gr.Radio(["Vector-Search", "GraphRAG"], label="RAG-method:")
    clear = gr.Button("Clear")


    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [chatbot, rag_method], chatbot
    )
    
    clear.click(lambda: None, None, chatbot, queue=False)

    
demo.queue()
demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


If you want to have the light-mode for the chatbot paste the following after the URL: /?__theme=light